# Conversion of h5 model to pb

This notebook is a part of Chapter-9, Scalable Inference Serving on Cloud with TensorFlow Serving and KubeFlow. We will convert a pretrained model, such as the ones we trained in previous chapters, to a `pb` model in order to use it for serving using TensorFlow Serving.

Please update the path of the `h5` model in `h5_model_path`.

The desired location and model name in `pb_model_path` can also be changed.

In [1]:
try:
    # Mount Google Drive
    from google.colab import drive

    drive.mount("/content/gdrive")

    IS_COLAB_ENV = True
except:
    IS_COLAB_ENV = False
IS_COLAB_ENV

Mounted at /content/gdrive


True

In [2]:
import tensorflow
from tensorflow import keras
from tensorflow.python.framework.convert_to_constants import (
    convert_variables_to_constants_v2,
)

from keras.models import load_model
import numpy as np
from keras.utils import CustomObjectScope
from keras import backend as K
import tensorflow as tf

In [3]:
# Edit the h5_model_path here as needed. We have tested this with the model trained in code/chapter-3/

h5_model_path = "model.h5"
if IS_COLAB_ENV:
    h5_model_path = "/content/gdrive/MyDrive/Practical-Deep-Learning-Book/code-outputs/chapter-3/model.h5"

tf_savedmodel_path = "model_tf"
if IS_COLAB_ENV:
    tf_savedmodel_path = "/content/gdrive/MyDrive/Practical-Deep-Learning-Book/code-outputs/chapter-9/model-tf"

pb_model_path = f"{tf_savedmodel_path}.pb"

print(h5_model_path)
print(tf_savedmodel_path)
print(pb_model_path)

/content/gdrive/MyDrive/Practical-Deep-Learning-Book/code-outputs/chapter-3/model.h5
/content/gdrive/MyDrive/Practical-Deep-Learning-Book/code-outputs/chapter-9/model-tf
/content/gdrive/MyDrive/Practical-Deep-Learning-Book/code-outputs/chapter-9/model-tf.pb


## **STEP 1:** Load the saved model, and convert the model to tensorflow's SavedModel format

In [4]:
with CustomObjectScope({"relu6": tf.nn.relu6}):
    model = load_model(h5_model_path)
    model.save(tf_savedmodel_path, save_format="tf")

### Verify the SavedModel

In [5]:
# Verify the SavedModel

# Load the model
loaded_model = tf.keras.models.load_model(tf_savedmodel_path)

# Check its architecture
loaded_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 mobilenet_1.00_224 (Functi  (None, 7, 7, 1024)        3228864   
 onal)                                                           
                                                                 
 global_average_pooling2d (  (None, 1024)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense (Dense)               (None, 64)                65600     
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 2)                 130   

## **STEP 2:** Convert SavedModel to .pb format

### Convert the model and obtain the frozen graph

In [6]:
full_model = tf.function(lambda x: loaded_model(x))
full_model = full_model.get_concrete_function(
    tf.TensorSpec(loaded_model.inputs[0].shape, loaded_model.inputs[0].dtype)
)

frozen_func = convert_variables_to_constants_v2(full_model)
frozen_func.graph.as_graph_def()

### Save the frozen graph to disk


In [7]:
tf.io.write_graph(
    graph_or_graph_def=frozen_func.graph,
    logdir="./frozen_models",
    name=pb_model_path,
    as_text=False,
)

'/content/gdrive/MyDrive/Practical-Deep-Learning-Book/code-outputs/chapter-9/model-tf.pb'